# Selecting Data Granularity Appropriate to the Hypothesis

##Azure Machine Learning workspace

It is a logical container for your machine learning experiments, compute target, datastore, machine learning models, docker images, deployed services...

Keeps them all together for teams to collaborate

### Create an Azure Machine learning workspace

In [5]:
# Check core SDK version number
import azureml.core
from azureml.core import Workspace
from azureml.core import Dataset

print('SDK version:', azureml.core.VERSION)

SDK version: 1.0.74

In [6]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace

try:
  ws = Workspace.create(
    name="your-workspace-name", 
    subscription_id='your-subscription-id', 
    resource_group='your-resource-group'
    location='your-preferred-location'
  )
  print('workspace configuration succeeded')
except:
  print('Workspace not found')
  
ws.name

###Use an existing workspace

In [8]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'your-subscription-id'
resource_group = 'your-resource-group'
workspace_name = 'your-workspace-name'

workspace = Workspace(subscription_id, resource_group, workspace_name)
workspace.name

Out[ 4 ]: 'customer_360_ws'

## Use Datasets directly in training

### Create a TabularDataset

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

Every workspace comes with a default datastore (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and create Dataset from it. We will now upload the Iris data to the default datastore (blob) within your workspace.

In [12]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
try:
  dataset = Dataset.get_by_name(workspace, name='Flight Delays Data')
  file = dataset.download(target_path='/train-dataset/flight_data', overwrite=True)
  print('File target path {}'.format(file))
except:
  print('Not completed')

File target path ['/train-dataset/flight_data/Flight Delays Data.csv']

In [14]:
datastore = workspace.get_default_datastore()
datastore.upload_files(files = file,
                       target_path = '/train-dataset/Flight Delays Data/',
                       overwrite = True,
                       show_progress = True)

Uploading an estimated of 1 files
Uploading /train-dataset/flight_data/Flight Delays Data.csv
Uploaded /train-dataset/flight_data/Flight Delays Data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
 Out[ 7 ]: $AZUREML_DATAREFERENCE_103a02d77c774dadb53c3b197c305d64

In [15]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, '/train-dataset/test_file/Flight Delays Data.csv')])

# preview the first 3 rows of the dataset
df = dataset.to_pandas_dataframe()
df.describe()

Out[ 8 ]: 
 Year Month DayofMonth DayOfWeek OriginAirportID DestAirportID CRSDepTime DepDelay DepDel15 CRSArrTime ArrDelay ArrDel15 Cancelled
count 2719418.0 2.719418e+06 2.719418e+06 2.719418e+06 2.719418e+06 2.719418e+06 2.719418e+06 2.691974e+06 2.691974e+06 2.719418e+06 2.690385e+06 2.719418e+06 2.719418e+06
mean 2013.0 6.979968e+00 1.579747e+01 3.898391e+00 1.274226e+04 1.274246e+04 1.326645e+03 1.053687e+01 2.023419e-01 1.505270e+03 6.637688e+00 2.166316e-01 1.067618e-02
std 0.0 1.984331e+00 8.799860e+00 1.985988e+00 1.501973e+03 1.501969e+03 4.713766e+02 3.609953e+01 4.017458e-01 4.939662e+02 3.864881e+01 4.119496e-01 1.027726e-01
min 2013.0 4.000000e+00 1.000000e+00 1.000000e+00 1.014000e+04 1.014000e+04 1.000000e+00 -6.300000e+01 0.000000e+00 1.000000e+00 -9.400000e+01 0.000000e+00 0.000000e+00
25% 2013.0 5.000000e+00 8.000000e+00 2.000000e+00 1.129200e+04 1.129200e+04 9.200000e+02 -4.000000e+00 0.000000e+00 1.120000e+03 -1.100000e+01 0.000000e+00 0.000000e+00
50% 2013.0 7.000000e+00 1.600000e+01 4.000000e+00 1.289200e+04 1.289200e+04 1.320000e+03 -1.000000e+00 0.000000e+00 1.528000e+03 -3.000000e+00 0.000000e+00 0.000000e+00
75% 2013.0 9.000000e+00 2.300000e+01 6.000000e+00 1.405700e+04 1.405700e+04 1.725000e+03 9.000000e+00 0.000000e+00 1.918000e+03 1.000000e+01 0.000000e+00 0.000000e+00
max 2013.0 1.000000e+01 3.100000e+01 7.000000e+00 1.537600e+04 1.537600e+04 2.359000e+03 1.863000e+03 1.000000e+00 2.359000e+03 1.845000e+03 1.000000e+00 1.000000e+00

###Basic statistical description of airlines

As a first step, we consider all the flights from all carriers. Here, the aim is to classify the airlines with respect to 
their punctuality and for that purpose, we compute a few basic statisticial parameters:

In [18]:
#__________________________________________________________________
# function that extract statistical parameters from a grouby objet:
def calc_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
  
  
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = df['DepDelay'].groupby(df['Carrier']).apply(calc_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

Out[ 9 ]: 
 count max mean min
Carrier 
HA 18636.0 1847.0 1.151052 -23.0
VX 34795.0 765.0 14.352407 -21.0
F9 35765.0 789.0 12.110275 -63.0
YV 51870.0 662.0 9.544920 -31.0
AS 68907.0 654.0 0.625365 -25.0
9E 78110.0 710.0 9.734131 -25.0
FL 92430.0 1113.0 10.185265 -49.0
MQ 109556.0 1244.0 15.538912 -60.0
B6 121764.0 537.0 12.621719 -30.0
EV 154007.0 838.0 14.489478 -52.0
OO 158537.0 773.0 7.882684 -45.0
US 233283.0 860.0 4.944338 -24.0
UA 285363.0 787.0 12.583842 -27.0
AA 288282.0 1863.0 12.064007 -28.0
DL 384402.0 1165.0 7.362576 -25.0
WN 576267.0 608.0 12.838167 -16.0